In [1]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, Dropdown


In [3]:
def generar_sistema(N, semilla=42):
    """
    Genera un sistema lineal Ax = b de tamaño NxN con A de rango completo
    y una solución ground truth x.
    
    Parámetros:
        N (int): Tamaño de la matriz cuadrada A.
        semilla (int): Semilla aleatoria para reproducibilidad.
    
    Retorna:
        A (ndarray): Matriz NxN de rango completo.
        x_true (ndarray): Vector solución ground truth.
        b (ndarray): Vector resultado (Ax).
    """
    rng = np.random.default_rng(semilla)
    
    # Generamos una matriz aleatoria y verificamos que tenga rango completo
    while True:
        A = rng.integers(low=-10, high=10, size=(N, N))
        if np.linalg.matrix_rank(A) == N:
            break
    
    # Generamos un x ground truth aleatorio
    x_true = rng.integers(low=-5, high=5, size=(N, 1))
    
    # Calculamos b = A @ x
    b = A @ x_true
    
    return A, x_true, b


N = 3
A, x_true, b = generar_sistema(N, semilla=123)
print("Matriz A:\n", A)
print("x (ground truth):\n", x_true)
print("b = Ax:\n", b)
print("Condición de A:", np.linalg.cond(A))

Matriz A:
 [[-10   3   1]
 [ -9   8  -6]
 [ -5  -7  -4]]
x (ground truth):
 [[-4]
 [-2]
 [ 3]]
b = Ax:
 [[37]
 [ 2]
 [22]]
Condición de A: 3.006896797645594


---

# GMRes Profe

In [13]:
def GMRes(A, b, x0=np.array([0.0]), m=10, flag_display=True, threshold=1e-12):
    n = len(b)
    if len(x0)==1:
        x0=np.zeros(n)
    r0 = b - np.dot(A, x0)
    nr0=np.linalg.norm(r0)
    out_res=np.array(nr0)
    Q = np.zeros((n,min(n,m+1)))
    H = np.zeros((min(n,m+1),min(n,m)))
    print(r0)
    print(nr0)
    Q[:,0] = r0 / nr0
    flag_break=False
    for k in np.arange(min(m,n)):
        y = np.dot(A, Q[:,k])
        if flag_display:
            print('||y||=',np.linalg.norm(y))
        for j in np.arange(k+1):
            H[j][k] = np.dot(Q[:,j], y)
            if flag_display:
                print('H[',j,'][',k,']=',H[j][k])
            y = y - np.dot(H[j][k],Q[:,j])
            if flag_display:
                print('||y||=',np.linalg.norm(y))
        # All but the last equation are treated equally. Why?
        if k+1<n:
            H[k+1][k] = np.linalg.norm(y)
            if flag_display:
                print('H[',k+1,'][',k,']=',H[k+1][k])
            if (np.abs(H[k+1][k]) > 1e-16):
                Q[:,k+1] = y/H[k+1][k]
            else:
                print('flag_break has been activated')
                flag_break=True
            # Do you remember e_1? The canonical vector.
            e1 = np.zeros((k+1)+1)        
            e1[0]=1
            H_tilde=H[0:(k+1)+1,0:k+1]
        else:
            H_tilde=H[0:k+1,0:k+1]
        # Solving the 'SMALL' least square problem. 
        # This could be improved with Givens rotations!
        ck = np.linalg.lstsq(H_tilde, nr0*e1,rcond=None)[0] 
        if k+1<n:
            x = x0 + np.dot(Q[:,0:(k+1)], ck)
        else:
            x = x0 + np.dot(Q, ck)
        # Why is 'norm_small' equal to 'norm_full'?
        norm_small=np.linalg.norm(np.dot(H_tilde,ck)-nr0*e1)
        out_res = np.append(out_res,norm_small)
        if flag_display:
            norm_full=np.linalg.norm(b-np.dot(A,x))
            print(r'..........||b-A\,x_k||=',norm_full)
            print(r'..........||H_k\,c_k-nr0*e1||',norm_small);
        if flag_break:
            if flag_display: 
                print('EXIT: flag_break=True')
            break
        if norm_small<threshold:
            if flag_display:
                print('EXIT: norm_small<threshold')
            break
    return x, out_res

x0 = np.zeros(N)
x_our, _ = GMRes(A,b,x0)


[[-35. -35. -35. -35.]
 [-38. -38. -38. -38.]
 [-23. -23. -23. -23.]
 [  1.   1.   1.   1.]]
113.11940593903418


ValueError: could not broadcast input array from shape (4,4) into shape (4,)

# GMRes con HouseHolder

In [ ]:
# householder reflection
def householder_reflection(v):
    """
    Devuelve la matriz de reflexión de Householder para un vector v.

    Parámetros
    ----------
    v : ndarray (n,), vector de entrada.

    Retorna
    -------
    H : ndarray (n,n), matriz de reflexión de Householder.
    """
    v = v.reshape(-1, 1)  # Aseguramos que v sea columna
    I = np.eye(v.shape[0])
    return I - 2 * (v @ v.T) / (v.T @ v)

def getHessenbergMatrix(A,b):
    """
    Reduce una matriz A a su forma de Hessenberg utilizando reflexiones de Householder.
    Parámetros
    ----------
    A : ndarray (n,n), matriz de entrada.
    b : ndarray (n,), vector de resultados.
    Retorna
    -------
    H : ndarray (n,n), matriz de Hessenberg.
    """
    N = A.shape[0]
    e1 = np.zeros((N, 1))
    e1[0] = 1
    # Primer vector
    vec1 = b / np.linalg.norm(b)
    alpha1 = -np.sign(b[0]) * np.linalg.norm(b)
    u1 = (b - alpha1 * e1) / np.linalg.norm(b - alpha1 * e1)
    
    #-------------------------------
    
    # segundo vector
    vec2 = A @ vec1
    # print("vec2:\n", vec2)
    # 1. Aplicamos las H's anteriores
    H1 = householder_reflection(vec1)
    aux2 = H1 @ vec2
    # quitar el primer elemento de aux2
    # print("aux2 antes de quitar el elemento\n",aux2)
    aux2 = aux2[1:]
    # print("aux2 despues de quitar el elemtno\n",aux2)
    alpha_2 = -np.sign(aux2[0]) * np.linalg.norm(aux2)
    u2 = (aux2 - alpha_2 * e1[:-1]) / np.linalg.norm(aux2 - alpha_2 * e1[:-1])
    # print("u2:\n", u2)

    #-------------------------------
    # tercer vector
    vec3 = A @ vec2
    print("vec3:\n", vec3)
    # 1. Aplicamos las H's anteriores
    H2_hat = householder_reflection(u2)
    H2 = np.block([
    [np.ones((1,1)), np.zeros((1, H2_hat.shape[1]))],
    [np.zeros((H2_hat.shape[0], 1)), H2_hat]])
    # print("H1:")
    # print(H1)
    # print("H2:")
    # print(H2)
    aux3 = H2 @ H1 @ vec3
    # quitar los primer 2 elementos de aux
    aux3 = aux3[2:]
    alpha_3 = -np.sign(aux3[0]) * np.linalg.norm(aux3)
    u3 = (aux3 - alpha_3 * e1[:-2]) / np.linalg.norm(aux3 - alpha_3 * e1[:-2])
    # u3 = np.vstack(([[0.],[0.]], u3))

    #--------------------------------
    # matriz de Hessenberg
    # print("u3:\n", u3)
    # H3_hat = householder_reflection(u3)
    # print("H3_hat:\n", H3_hat)
    H3 = np.block([
    [np.eye(2), np.zeros((2,1))],
    [np.zeros((1,2)), u3]
    ])
    print("H1:\n", H1)
    print("H2:\n", H2)
    print("H3 :\n", H3)
    Q = H3 @ H2 @ H1
    Hessenberg = Q @ A

    u_vectors = [u1, u2, u3]

    return Q, Hessenberg, u_vectors

Q, H, u_vectors = getHessenbergMatrix(A,b)

print("Matriz Q:")
print(Q)
print("Matriz de Hessenberg H:")
print(H)


vec3:
 [[ 41.44532053]
 [ 28.03244524]
 [139.25720519]]
H1:
 [[-0.47442111 -0.07969844 -0.87668282]
 [-0.07969844  0.99569198 -0.04738826]
 [-0.87668282 -0.04738826  0.47872913]]
H2:
 [[ 1.          0.          0.        ]
 [ 0.         -0.91115286  0.41206853]
 [ 0.          0.41206853  0.91115286]]
H3 :
 [[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
Matriz Q:
[[-0.47442111 -0.07969844 -0.87668282]
 [-0.28863594 -0.9267548   0.24044716]
 [-0.83163328  0.36711538  0.41666821]]
Matriz de Hessenberg H:
[[ 9.84491115  4.07592892  3.51050081]
 [10.02491681 -9.96307632  4.31010422]
 [ 2.92895332 -2.47465424 -4.70099837]]


# Debuggeando a lo primitivo

1. **¿Q es unitaria?**

In [5]:
print(np.dot(Q[0],Q[1]))
print(np.dot(Q[0],Q[2]))
print(np.dot(Q[1],Q[2]))

8.326672684688674e-17
1.734723475976807e-16
0.0


Sí, $Q$ es unitaria

---

**2. Construcción del primer vector de Hessenberg**




In [12]:
u1, u2, u3 = u_vectors

In [13]:
vec1 = b / np.linalg.norm(b)
print("vec1:\n", vec1)
H1 = householder_reflection(u1)
print("H1 @ vec1:\n", H1 @ vec1)

vec1:
 [[0.85860966]
 [0.04641133]
 [0.51052466]]
H1 @ vec1:
 [[-1.00000000e+00]
 [-8.67361738e-18]
 [-5.55111512e-17]]


funciona 👌

3. segundo vector

In [19]:
vec2 = A @ vec1
print("vector 2:\n", vec2)
H2_hat = householder_reflection(u2)
print("H2_hat:\n", H2_hat)
# hay que agregarle la idententida para que sea 3x3
H2 = np.block([
    [np.ones((1,1)), np.zeros((1, H2_hat.shape[1]))],
    [np.zeros((H2_hat.shape[0], 1)), H2_hat]])
print("H2:\n",H2)
print("H2 @ H1 @ vec2:\n",  H2 @ H1 @ vec2)

vector 2:
 [[ -7.93633797]
 [-10.4193443 ]
 [ -6.66002631]]
H2_hat:
 [[-0.91115286  0.41206853]
 [ 0.41206853  0.91115286]]
H2:
 [[ 1.          0.          0.        ]
 [ 0.         -0.91115286  0.41206853]
 [ 0.          0.41206853  0.91115286]]
H2 @ H1 @ vec2:
 [[10.69789984]
 [ 8.43441501]
 [-5.50633535]]


🫠😭🫠